In [1]:
from pyspark.sql import SparkSession
from pymongo import MongoClient
import urllib.parse
import pyspark.sql.functions as sqlf
from pyspark.ml import Pipeline
from pyspark.ml.feature import MinHashLSH, NGram, Tokenizer, CountVectorizer
from pyspark.sql.functions import col
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import monotonically_increasing_id

import os
os.chdir('/Users/martikvm/PycharmProjects/DoubleSearch/twitter')
from DisjointSet import DisjointSet
import sys
sys.setrecursionlimit(3000000)

all_saved_tweets = '/Users/martikvm/PycharmProjects/DoubleSearch/twitter/all_tweets_from_mongo_2019'
all_saved_tweets_file = '/Users/martikvm/PycharmProjects/DoubleSearch/twitter/all_tweets_2019.txt'
normalized_file = '/Users/martikvm/PycharmProjects/DoubleSearch/twitter/normalizedTweetsWithoutLinks_2019_test.txt'
#normalized_file = '/Users/martikvm/PycharmProjects/DoubleSearch/twitter/tweets_sample.txt'
results_file = '/Users/martikvm/PycharmProjects/DoubleSearch/twitter/results/TweetsAlike_2019_testreal.txt'
beautify_results_file = '/Users/martikvm/PycharmProjects/DoubleSearch/twitter/results/niceResults_2019_testreal.txt'


# os.environ["SPARK_HOME"] = "/Applications/spark/spark-2.3.1-bin-hadoop2.7"
os.environ["PYSPARK_PYTHON"] = "/Users/martikvm/anaconda3/envs/py35/bin/python"
# os.environ["PYSPARK_DRIVER_PYTHON"] = "/Users/martikvm/anaconda3/envs/py35/bin/python"

credPath = '/Users/martikvm/PycharmProjects/DoubleSearch/twitter/credentialsMongo.txt'
credFile = open(credPath, 'r')
loginMA = urllib.parse.quote_plus(credFile.readline().strip())
passwordMA = urllib.parse.quote_plus(credFile.readline()).strip()
client = MongoClient("mongodb+srv://" +
                     loginMA + ":" +
                     passwordMA +
                     "@doublesearchintwitter-m3qge.mongodb.net")
db = client.twitter_march2019
db_transformed_df2=client.test.transformed_df2
db_tweets_pairs = client.results_2019.tweets_pairs
db_tweets_pairs.create_index([('second_tweet', 1), ('first_tweet', 1)], unique = True)
tweets = db.tweets
allTweets = tweets.find(no_cursor_timeout=True).sort("text", 1)
keepAllTweets = []
spark = SparkSession \
    .builder \
    .master("local") \
    .appName("twitter") \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.11:2.3.0') \
    .config("spark.mongodb.input.uri", "mongodb+srv://" +
            loginMA + ":" +
            passwordMA +
            "@doublesearchintwitter-m3qge.mongodb.net/twitter_march2019.tweets") \
    .config("spark.mongodb.output.uri", "mongodb+srv://" +
            loginMA + ":" +
            passwordMA +
            "@doublesearchintwitter-m3qge.mongodb.net/") \
    .getOrCreate()

#df = spark.read.format("com.mongodb.spark.sql.DefaultSource").load()
#df.printSchema()
#df.select("text").write.text(all_saved_tweets)

In [ ]:
############## Очистка твитов
#import re
#
#with open(normalized_file, 'w') as output, open(all_saved_tweets_file, "r") as input:
#    print("Something")
#    for tweet in input:
#        tweet = tweet.strip()
#        oneTweet = re.sub("http(\S)+", "[link]", tweet)
#        oneTweet = re.sub("@(\S)+", "[user]", oneTweet)
#        output.write(oneTweet + '\n')

In [2]:
df1 = spark.read.text(normalized_file).withColumn("id", monotonically_increasing_id()) # .limit(500)

In [3]:
df1.count()

231279

In [ ]:
df1.show()

In [4]:
tokenizer = Tokenizer(inputCol="value", outputCol="words")

In [5]:
ngram = NGram(n=1, inputCol="words", outputCol="ngrams")

In [6]:
cv = CountVectorizer(inputCol="ngrams", outputCol="features")

In [7]:
pipeline = Pipeline(stages=[tokenizer, ngram, cv])
model = pipeline.fit(df1)
df2 = model.transform(df1)

In [ ]:
df2.show()

In [8]:
def getsparsesize(v):
    return v.values.size


getsize_udf = udf(getsparsesize, IntegerType())
df2_with_lengths = df2.select("id", "value", "features", getsize_udf("features").alias("vec_size"))
df2_with_lengths.show()

df2NotNull = df2_with_lengths.filter(getsize_udf(df2["features"]) != 0)

mh = MinHashLSH(inputCol="features", outputCol="hashes", numHashTables=128)
model2 = mh.fit(df2)
transformed_df2 = model2.transform(df2NotNull)
transformed_df2.show()

+---+--------------------+--------------------+--------+
| id|               value|            features|vec_size|
+---+--------------------+--------------------+--------+
|  0|"- Ох, Анечка, ес...|(262144,[1,3,5,6,...|      24|
|  1|твой путь (Владим...|(262144,[1,40,340...|       7|
|  2|[user] Я уж руки ...|(262144,[0,5,293,...|       6|
|  3|[user] [user] [us...|(262144,[0,1,3,7,...|      12|
|  4|[user] У меня нет...|(262144,[0,2,14,2...|       9|
|  5|  [user] Так что, да|(262144,[0,17,41,...|       4|
|  6|[user] [user] [us...|(262144,[0,4,1959...|       5|
|  7|Ещё одна Юля, тол...|(262144,[36,45,88...|       6|
|  8|[user] Я перемаза...|(262144,[0,1,3,5,...|      17|
|  9|[user] Это что ща...|(262144,[0,7,10,7...|       5|
| 10|чувак хочет кожан...|(262144,[26,272,1...|       6|
| 11|                ОКЕЙ|(262144,[1145],[1...|       1|
| 12|сука знаете что к...|(262144,[5,7,8,10...|      19|
| 13|Украинцев ждет но...|(262144,[1,8,13,1...|      11|
| 14|    #Здоровье [link]|(2621

In [ ]:
transformed_df2.select("id", "value").write.format("com.mongodb.spark.sql.DefaultSource").mode("append").option("database", "test").option("collection", "transformed_df2").save()

In [ ]:
#for k in range(0, df1.count()):
#    edges.append(k)
#print(max_tweet_id)
#print(df1.count())
#print(edges)

In [9]:
def colour_tweets(rows_with_ids, edges):
    for row in rows_with_ids:
        print(row)
        ids = row["ids"]
        for x in ids[1:]:
            #edges.union(ids[0], x)
            try:
                db_tweets_pairs.insert_one({'first_tweet': ids[0], 'second_tweet': x})
            except Exception:
                pass

In [ ]:
#for debug
#def colour_tweets(rows_with_ids, edges):
#    new_colour = 1
#
#    for row in rows_with_ids:
#        flag = True
#        print(row)
#        for id in row["ids"]:
#            to_change = []
#            if (flag):
#                new_colour = id
#                flag = False
#                if (edges[id] != id):
#                    new_colour = edges[id]
#            else:
#                to_change.append(edges[id])
#            edges[id] = new_colour
#            for i in range(0, len(edges)):
#                if edges[i] == id:
#                    edges[i] = new_colour
#            for k in to_change:
#                for i in range(0, len(edges)):
#                    if (edges[i] == k):
#                        edges[i] = new_colour

In [ ]:
def getHashColumns(df0, x):
    sum_of_hashes = 0
    for y in range(x, x + 4):
        sum_of_hashes += int(df0[y][0])
    return sum_of_hashes
                    

gethashsums_udf = udf(getHashColumns)

max_tweet_id = df1.agg({"id": "max"}).collect()[0][0]
edges = DisjointSet(max_tweet_id + 1)

with open(results_file, 'w') as outf:
    for k in range(0, 128, 4):
        print("k = ", k)
        outf.write("============================\nk = " + str(k) + "\n============================\n")
        
        df3 = transformed_df2.select("value", "id", gethashsums_udf("hashes", sqlf.lit(k)).alias("hashes03")).groupBy(
            "hashes03").agg(sqlf.count('*').alias("num_tweets"), sqlf.collect_list("value").alias("tweets_texts"), sqlf.collect_list("id").alias("ids")).filter(
            col("num_tweets") > 1)
        df3.show()
             
        colour_tweets(df3.select("ids").collect(), edges)
        print("We coloured some tweets! Way to go!")
                            
        for row in df3.collect():
            for oneTweet in row.tweets_texts:
                outf.write(oneTweet + '\n')
            outf.write(str(row.num_tweets))
            outf.write('\n\n')
                            
        print('transformed')
        print()

k =  0
+-----------+----------+--------------------+--------------------+
|   hashes03|num_tweets|        tweets_texts|                 ids|
+-----------+----------+--------------------+--------------------+
|-2059049121|         2|[[user] ааааааааа...|      [6748, 212815]|
|-2158252352|         2|[Пре… [link], Пре...|    [131858, 142015]|
|-2369172556|         3|[тупо я, тупо я, ...|[171530, 195652, ...|
|-2664765492|         2|[Мисс киша, Мисс ...|      [91554, 91555]|
|-2736158904|         2|[День 5. 7:43, Де...|      [23184, 71910]|
|-3641690418|         2|[О перспективах п...|    [159620, 159627]|
|-3748229908|         2|[шу: ок [link], О...|       [1104, 28671]|
|-3867356825|         2|[[user] Там неудо...|    [136804, 224009]|
|-4584906994|         2|[[user] Выдать кл...|    [121429, 122023]|
|-4590100057|         2|[(я не готова, на...|    [142219, 174746]|
|-4640823169|         2|[Главный бухгалте...|      [47218, 50174]|
|-4650176327|         2|[Заходи и смотри ...|       [30

Row(ids=[71603, 90463])
Row(ids=[23126, 227984])
Row(ids=[49201, 53958, 56265, 87097, 118865, 122920, 139214, 160673, 164992, 198333, 204568, 208370, 210910, 218998, 226418])
Row(ids=[185235, 225596])
Row(ids=[97894, 216819])
Row(ids=[17610, 84427, 167317])
Row(ids=[123026, 132172, 213004])
Row(ids=[174331, 174333])
Row(ids=[6571, 133801])
Row(ids=[12569, 167521])
Row(ids=[6132, 40459, 57083, 63204, 107536, 127000, 129454, 134324, 201664])
Row(ids=[199778, 199808])
Row(ids=[111020, 205042])
Row(ids=[34749, 48233])
Row(ids=[8280, 86955])
Row(ids=[72006, 129542, 195911])
Row(ids=[22360, 133808, 213807])
Row(ids=[19866, 123047])
Row(ids=[137385, 157070])
Row(ids=[222175, 222176])
Row(ids=[30247, 115043, 154124, 190518])
Row(ids=[173014, 188082])
Row(ids=[101930, 169996])
Row(ids=[12755, 33696])
Row(ids=[25887, 67904, 94528, 100994, 104076, 150911, 183119, 199222, 211608, 225010])
Row(ids=[29072, 29080])
Row(ids=[160604, 160770])
Row(ids=[96206, 113554, 119728])
Row(ids=[138552, 228391])
R

Row(ids=[20112, 21649])
Row(ids=[158100, 217905])
Row(ids=[159332, 194085, 208642])
Row(ids=[42028, 178746])
Row(ids=[66789, 67110, 67334])
Row(ids=[115284, 148423])
Row(ids=[143536, 143551])
Row(ids=[146640, 146761])
Row(ids=[120953, 229649])
Row(ids=[129762, 203205])
Row(ids=[99788, 213526])
Row(ids=[3298, 3724])
Row(ids=[1917, 172284])
Row(ids=[3386, 5846, 7865, 11571, 13248, 13363, 17078, 31714, 32201, 34208, 43638, 46970, 47006, 57392, 62421, 80319, 87188, 88799, 92695, 96784, 97310, 97497, 102088, 106269, 111136, 112241, 121045, 121819, 124877, 133509, 136974, 139582, 139724, 143175, 143470, 145331, 151003, 152035, 153802, 159965, 160040, 160987, 161951, 163776, 167804, 168012, 169338, 172321, 174277, 174398, 178180, 179436, 182694, 183297, 186166, 192684, 192859, 193237, 198829, 199451, 200455, 202037, 203384, 204822, 207122, 207591, 207863, 209076, 212813, 214593, 214769, 219992, 220012, 231243])
Row(ids=[41950, 77597])
Row(ids=[126862, 228841])
Row(ids=[29060, 29067, 126542, 1

Row(ids=[130253, 130487])
Row(ids=[52421, 201772])
Row(ids=[42209, 80546, 82470, 82646, 86941, 200181])
Row(ids=[184801, 228751])
Row(ids=[77327, 151404])
Row(ids=[150325, 177397])
Row(ids=[67250, 67690])
Row(ids=[146204, 221177, 228264])
Row(ids=[112099, 113488])
Row(ids=[38825, 41369])
Row(ids=[51951, 202509])
Row(ids=[13042, 19007, 19008, 24348, 28677, 29323, 30069, 36113, 51914, 67884, 68702, 75665, 90424, 97530, 98172, 130334, 144788, 169823, 180402, 185620, 193493, 217964, 218659, 220646, 231138])
Row(ids=[80994, 228095])
Row(ids=[127145, 131733])
Row(ids=[163435, 174350, 198292])
Row(ids=[85973, 110684, 146449])
Row(ids=[118137, 153638, 187664])
Row(ids=[76385, 78657])
Row(ids=[41258, 189424])
Row(ids=[70690, 70691, 199564])
Row(ids=[160725, 192758])
Row(ids=[46302, 49108])
Row(ids=[70607, 80058, 157566])
Row(ids=[13440, 58081, 74351, 79762, 82287, 106334, 119586, 123161, 129789, 140662, 148081, 166582, 196434, 218018, 222041, 229176])
Row(ids=[133768, 133793])
Row(ids=[48011, 7

Row(ids=[138174, 178941])
Row(ids=[35701, 43490])
Row(ids=[130194, 196368])
Row(ids=[169593, 169746])
Row(ids=[80182, 195262])
Row(ids=[43942, 70173, 86669, 109914, 220053])
Row(ids=[190965, 209384])
Row(ids=[156742, 179172])
Row(ids=[161539, 187700])
Row(ids=[84848, 120230, 129380, 218577])
Row(ids=[120644, 120677])
Row(ids=[89276, 196606])
Row(ids=[2648, 185679, 188575, 211123])
Row(ids=[76491, 97126])
Row(ids=[14696, 223865])
Row(ids=[84576, 84585, 84635, 84650, 84661, 84698])
Row(ids=[10041, 71048, 95154, 116325, 118088, 125234, 139500, 145025, 152572, 214032, 224501, 227253])
Row(ids=[48494, 112577])
Row(ids=[214815, 229135])
Row(ids=[102398, 179449])
Row(ids=[20664, 130030])
Row(ids=[151039, 163348])
Row(ids=[4887, 5851])
Row(ids=[11499, 16579, 31160, 34399, 37253, 45416, 56446])
Row(ids=[153481, 177195])
Row(ids=[37330, 38484, 183069])
Row(ids=[45785, 171920])
Row(ids=[28037, 59023, 70955, 124501, 203428, 229505])
Row(ids=[419, 9187])
Row(ids=[40750, 41038])
Row(ids=[6258, 22876

...

Row(ids=[39981, 158386])
Row(ids=[75794, 95383, 181562])
Row(ids=[52134, 52494])
Row(ids=[137014, 137333])
Row(ids=[207891, 218956])
Row(ids=[62784, 162964])
Row(ids=[25614, 25734, 25815])
Row(ids=[103367, 113277])
Row(ids=[61481, 82878, 86039])
Row(ids=[19167, 32542, 187230])
Row(ids=[77872, 96779, 118996, 205258])
Row(ids=[13042, 19007, 19008, 24348, 28677, 29323, 30069, 36113, 51914, 67884, 68702, 75665, 90424, 97530, 98172, 130334, 144788, 169823, 180402, 185620, 193493, 217964, 218659, 220646, 231138])
Row(ids=[124303, 135284])
Row(ids=[27732, 48042, 89143, 124140, 228420])
Row(ids=[14829, 74253])
Row(ids=[72617, 109957])
Row(ids=[16684, 176591])
Row(ids=[12388, 39032, 101380, 106059, 114293, 136942, 141283, 143074, 166986, 224391])
Row(ids=[190412, 221227])
Row(ids=[59110, 230821])
Row(ids=[75616, 124042, 184908, 229378])
Row(ids=[65440, 182552, 226745])
Row(ids=[220301, 220544])
Row(ids=[37110, 88338])
Row(ids=[22393, 88916, 129709, 130844, 149482, 154685])
Row(ids=[10868, 12066

Row(ids=[77069, 81840, 188677, 206936])
Row(ids=[91069, 174606])
Row(ids=[75846, 76049])
Row(ids=[58062, 58184])
Row(ids=[79371, 123437])
Row(ids=[16389, 172482])
Row(ids=[83448, 168364])
Row(ids=[104038, 104149, 104269])
Row(ids=[144389, 154206])
Row(ids=[47970, 173601])
Row(ids=[140964, 140965, 159919, 159920, 160167, 160168, 224420])
Row(ids=[125863, 126187])
Row(ids=[28207, 38591, 40950, 59266, 62141, 62454, 62747, 63733, 64943, 72295, 80225, 92451, 103967, 106961, 119361, 133736, 137230, 144372, 148465, 154303, 197827, 207308, 226776])
Row(ids=[1291, 38656, 49900, 61934, 85112, 91675, 100515, 121401, 139248, 161970, 190397, 203233, 218819, 230925])
Row(ids=[61686, 123127, 123200])
Row(ids=[60583, 72773, 73627, 74472, 212175])
Row(ids=[105628, 106785])
Row(ids=[83249, 156928, 162559, 193879, 201954])
Row(ids=[161324, 181753])
Row(ids=[60990, 94588, 155204])
Row(ids=[25336, 204383])
Row(ids=[176261, 218483])
Row(ids=[59505, 62644, 64184, 76019, 90482, 130601])
Row(ids=[27433, 32267,

Row(ids=[82730, 158929])
Row(ids=[50249, 51476, 74637])
Row(ids=[88625, 197259])
Row(ids=[29662, 95784])
Row(ids=[111893, 112441, 139054, 208569])
Row(ids=[82246, 132913])
Row(ids=[126788, 126990, 127150, 128175])
Row(ids=[160320, 160497])
Row(ids=[139764, 141506])
Row(ids=[86359, 220141])
Row(ids=[5455, 36462])
Row(ids=[61141, 140567])
Row(ids=[28208, 134897])
Row(ids=[9661, 110366])
Row(ids=[24650, 206090])
Row(ids=[23001, 69682, 81959, 132850, 168643, 199391, 224710])
Row(ids=[167287, 179215])
Row(ids=[44698, 224105])
Row(ids=[52917, 101125, 206382])
Row(ids=[103022, 105929])
Row(ids=[119781, 120389])
Row(ids=[105187, 169564])
Row(ids=[13418, 130609])
Row(ids=[8977, 55463, 140060])
Row(ids=[9122, 35070, 35706, 35707, 53480])
Row(ids=[77357, 77366])
Row(ids=[20782, 213613])
Row(ids=[54708, 54844])
Row(ids=[47617, 147760])
Row(ids=[18192, 41785])
Row(ids=[12840, 163910])
Row(ids=[39352, 113187, 132692, 136121])
Row(ids=[50422, 50597])
Row(ids=[14285, 20100, 77548, 84250, 153194])
Row(

Row(ids=[127320, 220113])
Row(ids=[161997, 212076])
Row(ids=[82643, 193529, 223234])
Row(ids=[45915, 45919])
Row(ids=[7024, 35413, 123559, 225052])
Row(ids=[175483, 175503])
Row(ids=[60663, 226121])
Row(ids=[220995, 221063])
Row(ids=[152417, 189731, 189978])
Row(ids=[143342, 152661])
Row(ids=[126467, 126744, 129866])
Row(ids=[101795, 130735, 149444])
Row(ids=[166209, 167302])
Row(ids=[11339, 11350])
Row(ids=[59835, 90694])
Row(ids=[48242, 98125])
Row(ids=[52476, 64415, 70766, 100695, 104083, 131988, 198301, 207241])
Row(ids=[92613, 198173])
Row(ids=[53905, 122679])
Row(ids=[86848, 118199, 223918])
Row(ids=[59443, 113107, 126476, 147546])
Row(ids=[34302, 153083])
Row(ids=[75988, 97973])
Row(ids=[79921, 79944])
Row(ids=[130553, 151884])
Row(ids=[32348, 172824, 173729])
Row(ids=[164328, 224292])
Row(ids=[59213, 180379])
Row(ids=[119023, 119104])
Row(ids=[13436, 28031])
Row(ids=[119543, 136494, 139728])
Row(ids=[69093, 121323])
Row(ids=[157946, 158371])
Row(ids=[91264, 105740])
Row(ids=[39

Row(ids=[26998, 35506, 207611])
Row(ids=[14203, 22674, 48407, 55185, 95241, 119949, 171099, 211833, 222502])
Row(ids=[21143, 34004])
Row(ids=[23420, 99620, 147293])
Row(ids=[128570, 190541])
Row(ids=[53214, 62318, 78484, 84507])
Row(ids=[78447, 179546, 191288, 217813])
Row(ids=[90815, 156109])
Row(ids=[98361, 138978])
Row(ids=[51744, 118495, 146876])
Row(ids=[11981, 11982, 141591, 221821])
Row(ids=[139772, 147783])
Row(ids=[80031, 84407])
Row(ids=[28159, 28161])
Row(ids=[47072, 47605])
Row(ids=[129332, 177326])
Row(ids=[9075, 19113, 135430, 217227])
Row(ids=[5205, 56034, 108040, 127986, 200404, 207019])
Row(ids=[153446, 153585, 153700])
Row(ids=[107813, 191925])
Row(ids=[130397, 130399])
Row(ids=[911, 914, 9018, 9753, 11044, 15980, 16807, 16808, 17505, 30949, 30950, 37898, 52590, 53123, 53366, 57227, 60436, 70663, 70664, 78938, 82472, 88505, 88507, 88509, 90819, 92290, 92292, 92428, 92432, 95148, 103837, 105402, 113936, 119239, 129049, 131870, 138271, 143352, 157073, 162116, 162531, 17

In [ ]:
#TODO
#db_tweets_pairs.find().sort([('first_tweet', 1)])
with open(beautify_results_file, 'w') as out:
    colours = set(edges.colors_list())
    colours = sorted(colours)
    for colour in colours:
        print(colour)
        out.write("\n=== " + str(colour) + " ===\n")
        for k in range(len(edges.colors_list())):
            print(k)
            if edges.colors_list()[k] == colour:
                out.write(db_transformed_df2.find_one({"id": k})['value'])
                out.write('\n')

In [ ]:
print(edges.colors_list())

In [ ]:
list(db_tweets_pairs.find({'first_tweet': 15}))

In [ ]:
sys.setrecursionlimit(3000000)